# Wildfire Disaster Predictor (Mini Dataset 500 Samples)
Uses classification and regression with ML & DL (ANN, CNN, RNN)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM
from google.colab import files
import os


In [ ]:
dataset_name = 'wildfire_risk_dataset_500.csv'
if not os.path.exists(dataset_name):
    uploaded = files.upload()
    for fn in uploaded:
        if fn.endswith('.csv'):
            dataset_name = fn

df = pd.read_csv(dataset_name)
df.head()

In [ ]:
le_activity = LabelEncoder()
le_risk = LabelEncoder()
df['Human Activity Level'] = le_activity.fit_transform(df['Human Activity Level'])
df['Fire Risk Level'] = le_risk.fit_transform(df['Fire Risk Level'])

X = df.drop('Fire Risk Level', axis=1)
y_class = df['Fire Risk Level']
y_reg = df['Drought Index']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train_class, y_test_class = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42)
_, _, y_train_reg, y_test_reg = train_test_split(X_scaled, y_reg, test_size=0.2, random_state=42)


In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train_class)
print('Logistic Regression accuracy:', lr.score(X_test, y_test_class))

rf = RandomForestClassifier()
rf.fit(X_train, y_train_class)
print('Random Forest accuracy:', rf.score(X_test, y_test_class))


In [ ]:
num_classes = len(np.unique(y_class))
ann = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])
ann.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_ann = ann.fit(X_train, y_train_class, epochs=15, batch_size=16, validation_split=0.1, verbose=1)


In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train_reg)
y_pred_lin = lin_reg.predict(X_test)
print('Linear Regression MSE:', mean_squared_error(y_test_reg, y_pred_lin))
print('Linear Regression MAE:', mean_absolute_error(y_test_reg, y_pred_lin))

dnn = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])
dnn.compile(optimizer='adam', loss='mse', metrics=['mae'])
history_dnn = dnn.fit(X_train, y_train_reg, epochs=15, batch_size=16, validation_split=0.1, verbose=1)


In [ ]:
X_cnn = X_scaled.reshape(-1, X.shape[1], 1)
cnn = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X.shape[1],1)),
    MaxPooling1D(2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_cnn = cnn.fit(X_cnn, y_class, epochs=15, batch_size=16, validation_split=0.1, verbose=1)


In [ ]:
X_rnn = X_scaled.reshape(-1, X.shape[1], 1)
rnn = Sequential([
    LSTM(32, input_shape=(X.shape[1],1)),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])
rnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_rnn = rnn.fit(X_rnn, y_class, epochs=15, batch_size=16, validation_split=0.1, verbose=1)


In [ ]:
# Evaluate ANN classification
y_pred = np.argmax(ann.predict(X_test), axis=1)
print('ANN classification report:')
print(classification_report(y_test_class, y_pred))
plt.figure()
sns.heatmap(confusion_matrix(y_test_class, y_pred), annot=True, fmt='d')
plt.title('ANN Confusion Matrix')
plt.show()

# Regression evaluation for DNN
y_pred_dnn = dnn.predict(X_test).flatten()
print('DNN Regression MSE:', mean_squared_error(y_test_reg, y_pred_dnn))
print('DNN Regression MAE:', mean_absolute_error(y_test_reg, y_pred_dnn))

# Plot training curves
plt.figure()
plt.plot(history_ann.history['loss'], label='ann loss')
plt.plot(history_ann.history['val_loss'], label='ann val_loss')
plt.legend(); plt.title('ANN Loss')

plt.figure()
plt.plot(history_dnn.history['loss'], label='dnn loss')
plt.plot(history_dnn.history['val_loss'], label='dnn val_loss')
plt.legend(); plt.title('DNN Loss')


In [ ]:
ann.save('ann_classifier.h5')
dnn.save('dnn_regressor.h5')
